In [1]:
import numpy as np
from neural_network.layer import *
from neural_network.net import *
from queue import Queue
import matplotlib.pyplot as plt
from tqdm import tqdm
from neural_network.trainutils import *
import os

In [2]:
current_dir = os.getcwd()
face_dir = os.path.join(current_dir, "cmu+face+images", "faces")
dataset = Dataset(face_dir, flatten=False)

In [3]:
dataset.data.shape

(624, 120, 128)

In [4]:
train_dataset, _, test_dataset = train_test_split(dataset, (0.8, 0.0, 0.2))

In [5]:
len(train_dataset)

499

In [6]:
dataset[500][0].shape

(1, 1, 120, 128)

In [7]:
input_shape = train_dataset[0][0].shape
input_shape

(1, 1, 120, 128)

In [8]:
train_dataset[0]

(array([[[[ 0.89356498,  0.88204423,  0.70186609, ..., -0.07287527,
           -0.14926768,  0.06709001],
          [ 0.87623038,  0.85418384,  0.7840018 , ..., -0.08577082,
           -0.10870554,  0.16428764],
          [ 1.06071743,  1.06291336,  1.05501475, ...,  0.01652978,
           -0.20427228, -0.32743861],
          ...,
          [-0.96958661, -1.06568854, -0.90124237, ..., -0.79624408,
           -0.80406965, -0.78658602],
          [-0.96819614, -1.03387179, -0.93275801, ..., -0.78640354,
           -0.76315872, -0.76266749],
          [-0.84390258, -0.84255993, -0.55252975, ..., -0.81278744,
           -0.80262334, -0.78879583]]]]),
 array([[0.],
        [1.],
        [0.],
        [0.]]))

In [9]:
kernel_shape = np.ndarray([1, 1, 3, 3])
pad = 1
stride = 1

conv1 = Conv2D(kernel_shape, pad, stride, layer_id=1) # 120x128
maxpool1 = MaxPool2D(2, 0, 2, conv1) # 60x64
relu1 = ReLU(maxpool1)

conv2 = Conv2D(kernel_shape, pad, stride, relu1, layer_id=2)
maxpool2 = MaxPool2D(2, 0, 2, conv2) # 30x32
relu2 = ReLU(maxpool2)

conv3 = Conv2D(kernel_shape, pad, stride, relu2, layer_id=3)
maxpool3 = MaxPool2D(2, 0, 2, conv3) # 15x16
relu3 = ReLU(maxpool3)

flatten = Flatten(relu3)

linear = Linear(15*16, len(dataset.keys), flatten)
softmax_layer = Softmax(linear)
loss_layer = CrossEntropy(softmax_layer)

model = NeuralNetwork(conv1, softmax_layer, loss_layer)

In [10]:
kernel_shape.shape

(1, 1, 3, 3)

In [11]:
print("Forward order:")
for layer in model._forward_order:
    print(layer)

print("\nBackward order:")
for layer in model._backward_order:
    print(layer)

Forward order:
Conv2D 1: (1, 1, 3, 3), pad=1, stride=1
MaxPool2D: (2, 2), pad=0, stride=2
ReLU
Conv2D 2: (1, 1, 3, 3), pad=1, stride=1
MaxPool2D: (2, 2), pad=0, stride=2
ReLU
Conv2D 3: (1, 1, 3, 3), pad=1, stride=1
MaxPool2D: (2, 2), pad=0, stride=2
ReLU
Flatten
Linear: (240, 1) -> (4, 1)
Softmax
CrossEntropy

Backward order:
CrossEntropy
Softmax
Linear: (240, 1) -> (4, 1)
Flatten
ReLU
MaxPool2D: (2, 2), pad=0, stride=2
Conv2D 3: (1, 1, 3, 3), pad=1, stride=1
ReLU
MaxPool2D: (2, 2), pad=0, stride=2
Conv2D 2: (1, 1, 3, 3), pad=1, stride=1
ReLU
MaxPool2D: (2, 2), pad=0, stride=2
Conv2D 1: (1, 1, 3, 3), pad=1, stride=1


In [15]:
results = k_fold_cross_validation(10, model, train_dataset, epochs=50, lr=1e-3, validation_period=5, seed=585)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
plt.plot(loss_array[:50])
plt.title("Average loss per epoch")
plt.xlabel("Epoch")
plt.ylabel("Average Loss")
plt.grid()

NameError: name 'loss_array' is not defined

In [ ]:
unique, counts = np.unique(dataset.label, return_counts=True)

In [ ]:
print(dataset.keys[unique])
print(counts)

['left' 'right' 'straight' 'up']
[157 155 156 156]


In [ ]:
from PIL import Image
my_img = Image.open("test/woman_straight.jpg").convert("L")
my_img = np.array(my_img)#.reshape(-1, 1)
my_img.shape

AttributeError: 'Image' object has no attribute 'shape'

In [ ]:
model.forward(my_img)

ValueError: not enough values to unpack (expected 4, got 2)